In [3]:
import requests
import json
import pandas as pd
from requests_oauthlib import OAuth2Session #conda install requests-oauthlib

In [ ]:
#Dev credentials to run the API

client_secret="PFZeEax1Ln6qkFuqxCWTgum0o4O5sAaC"
client_id="3074457360917723621"
auth_link="https://miro.com/oauth/authorize"
redirect_uri="https://localhost:8080/miro"

## User authentication process

In [49]:
#Getting authentitation from other users

oauth = OAuth2Session(client_id, redirect_uri=redirect_uri)
authorization_url, state = oauth.authorization_url(
        auth_link)
print('Please go to %s and authorize access.' % authorization_url)

Please go to https://miro.com/oauth/authorize?response_type=code&client_id=3074457360917723621&redirect_uri=https%3A%2F%2Flocalhost%3A8080%2Fmiro&state=qSiLWfnci54Hx4nKw205muuLQGuKSq and authorize access.


After authorizing access, a new URL appears in the address bar. It should begin with https://localhost:8080/miro. Copy the URL, run the code block below, and paste it into the box that appears.

In [54]:
authorization_response = input('Copy and paste the URL address here:')
fetch_url="https://api.miro.com/v1/oauth/token?grant_type=authorization_code&code="+authorization_response[33: 39]+"&redirect_uri="+ redirect_uri+"&client_id="+client_id+"&client_secret="+ client_secret 

token = oauth.fetch_token(fetch_url,authorization_response=authorization_response, client_secret=client_secret)

Copy and paste the URL address here:https://localhost:8080/miro?code=gTHHOQ&state=qSiLWfnci54Hx4nKw205muuLQGuKSq&client_id=3074457360917723621&team_id=3074457360917633599


Using the info from the URL you pasted, the API took your information after authentication and created a token with your user_id and team_id so that it can now fetch data from your account.

In [57]:
token
user_access_token=token['access_token']
user_id=token['user_id']
access_token=token['access_token']
token

{'account_id': '3074457360917633599',
 'user_id': '3074457360917294320',
 'refresh_token': None,
 'expires_in': 0,
 'scope': ['boards:write', 'boards:read'],
 'access_token': 'z1hqk8yzDV3eFmJDhSS9i7dMrv4',
 'team_id': '3074457360917633599',
 'token_type': 'bearer',
 'expires_at': 1625612097.9835253}

In [64]:
#Create a new board
url = "https://api.miro.com/v1/boards"

payload = {
    "name": "Test Board",
    "sharingPolicy": {
        "access": "private",
        "teamAccess": "private"
    }
}
headers = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Authorization": "Bearer "+access_token
}

new_board_response = requests.request("POST", url, json=payload, headers=headers)
board_responsejson=json.loads(new_board_response.text)
print(new_board_response.text)

{
  "type" : "board",
  "team" : {
    "type" : "team",
    "name" : "Dev team",
    "id" : "3074457360917633599"
  },
  "currentUserConnection" : {
    "type" : "board-user-connection",
    "role" : "owner",
    "user" : {
      "type" : "user",
      "name" : "Sheldon Zhu",
      "id" : "3074457360917294320"
    },
    "id" : "3074457361078096658"
  },
  "modifiedAt" : "2021-07-06T23:08:13Z",
  "modifiedBy" : {
    "type" : "user",
    "name" : "Sheldon Zhu",
    "id" : "3074457360917294320"
  },
  "createdBy" : {
    "type" : "user",
    "name" : "Sheldon Zhu",
    "id" : "3074457360917294320"
  },
  "sharingPolicy" : {
    "access" : "private",
    "teamAccess" : "private",
    "accountAccess" : "private"
  },
  "viewLink" : "https://miro.com/app/board/o9J_l7zDaUg=",
  "name" : "Untitled",
  "id" : "o9J_l7zDaUg=",
  "owner" : {
    "type" : "user",
    "name" : "Sheldon Zhu",
    "id" : "3074457360917294320"
  },
  "description" : "",
  "picture" : null,
  "createdAt" : "2021-07-06

In [67]:
#Add widget to the board above(can be ran multiple times)

board_id=board_responsejson.get('id')
url = "https://api.miro.com/v1/boards/" + board_id+ "/widgets/"

#Go to https://developers.miro.com/reference#widget-types to see the different types of widgets, 
#then click on each widget to see what properties they have
payload = {
  "type": "card",
  "title": "Environmental effects of COVID-19",
    "x": 0,
    "y": 0,
  "style": {
    "backgroundColor": "#ff00ff"
  }
}
headers = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Authorization": "Bearer " + access_token
}

widget_response = requests.request("POST", url, json=payload, headers=headers)
widget_responsejson=json.loads(widget_response.text)
widget_id=widget_responsejson.get('id')
print(widget_response.text)

{
  "id" : "3074457361077538762",
  "type" : "card",
  "title" : "Environmental effects of COVID-19",
  "description" : "",
  "date" : null,
  "card" : null,
  "x" : 0.0,
  "rotation" : 0.0,
  "assignee" : null,
  "y" : 0.0,
  "scale" : 1.0,
  "style" : {
    "backgroundColor" : "#ff00ff"
  },
  "createdAt" : "2021-07-06T23:12:11Z",
  "createdBy" : {
    "type" : "user",
    "name" : "Sheldon Zhu",
    "id" : "3074457360917294320"
  },
  "modifiedAt" : "2021-07-06T23:12:11Z",
  "modifiedBy" : {
    "type" : "user",
    "name" : "Sheldon Zhu",
    "id" : "3074457360917294320"
  },
  "capabilities" : {
    "editable" : true
  }
}


In [82]:
#Get list of all widgets for a board with its respective board_id

url = "https://api.miro.com/v1/boards/" + board_id+ "/widgets/"

headers = {
    "Accept": "application/json",
    "Authorization": "Bearer "+ access_token
}

widgets_response = requests.request("GET", url, headers=headers)
widgets_responsejson=json.loads(widgets_response.text)
#Get the 0-th widget in the board
print(widgets_responsejson.get('data')[0])

IndexError: list index out of range